In [1]:
import os
import pandas as pd

from datetime import datetime

In [2]:
def tipologia_andre(x):
    
    if x in (-100, -50):
        return "A"
    elif x < 0:
        return "C"
    else:
        return "R"

In [16]:
columns = ['Data Contabile', 'Data', 'Addebiti', 'Accrediti', 'Dettagli',
       'Spesa Comune', 'Categoria']
source = "data/new_data/andre"
destination = "data/storico/andre"

filename = os.listdir(source)[0]

df = pd.read_excel(f"{source}/{filename}", header=11)
df.columns = columns
df["Addebiti"] = pd.to_numeric(df["Addebiti"].fillna("0").apply(lambda x: x.replace(".", "").replace(",", ".")))
df["Accrediti"] = pd.to_numeric(df["Accrediti"].fillna("0").apply(lambda x: x.replace(".", "").replace(",", ".")))
df["Importo"] = df["Accrediti"] - df["Addebiti"]
df.drop(columns=["Addebiti", "Accrediti", "Data Contabile"], inplace=True)
df["Data"] = pd.to_datetime(df["Data"])
df["Tipologia"] = df["Importo"].apply(tipologia_andre)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Data          94 non-null     datetime64[ns]
 1   Dettagli      94 non-null     object        
 2   Spesa Comune  31 non-null     object        
 3   Categoria     91 non-null     object        
 4   Importo       94 non-null     float64       
 5   Tipologia     94 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 4.5+ KB


In [35]:
def process_file_andre():
    
    columns = ['Data Contabile', 'Data', 'Addebiti', 'Accrediti', 'Dettagli',
       'Spesa Comune', 'Categoria']
    source = "data/new_data/andre"
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    destination = "data/storico/andre"

    filename = os.listdir(source)[0]

    df = pd.read_excel(f"{source}/{filename}", header=11)
    df.columns = columns
    df["Addebiti"] = pd.to_numeric(df["Addebiti"].fillna("0").apply(lambda x: x.replace(".", "").replace(",", ".")))
    df["Accrediti"] = pd.to_numeric(df["Accrediti"].fillna("0").apply(lambda x: x.replace(".", "").replace(",", ".")))
    df["Importo"] = df["Accrediti"] - df["Addebiti"]
    df.drop(columns=["Addebiti", "Accrediti", "Data Contabile"], inplace=True)
    df["Data"] = pd.to_datetime(df["Data"])
    df["Tipologia"] = df["Importo"].apply(tipologia_andre)
    
    append_data_to_original_andre(df)
    
    os.rename(f"{source}/{filename}", f"{destination}/{now}_{filename}")
    print(f"File {filename} moved from {source} to {destination}")
    
    return

In [22]:
# process_file_andre()

File ListaMovimenti (3).xlsx moved from data/new_data/andre to data/storico/andre


In [36]:
def append_data_to_original_andre(df):
    
    data_orig = "data/dataAndre.csv"
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    data_store = f"data/storico/dataAndre_{now}.csv"
    
    data_or = pd.read_csv(data_orig)

    data_f = pd.concat([data_or, df])
    
    assert data_or.shape[0] + df.shape[0] == data_f.shape[0], "Data leakage"
    
    os.rename(data_orig, data_store)
    
    data_f.to_csv(data_orig, index=False)
    
    print(f"File {data_orig} moved to {data_store}")
    
    return

In [37]:
process_file_andre()

File data/dataAndre.csv moved to data/storico/dataAndre_20210704103319.csv
File ListaMovimenti (3).xlsx moved from data/new_data/andre to data/storico/andre
